In [42]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [43]:
df = pd.read_csv('../data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [44]:
X = df.drop("math_score", axis=1)
Y= df["math_score"]
num_cols = [feature for feature in X.columns if df[feature].dtype != 'O']
cat_cols = [feature for feature in X.columns if df[feature].dtype == 'O']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
    ("oneHotEncoder", oh_transformer, cat_cols),
    ("StandardScaler", num_transformer, num_cols),
    ]
)


In [45]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

In [47]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [48]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "Random Forest": RandomForestRegressor(),
    "KNN": KNeighborsRegressor(),
    "XGBRegressor": XGBRegressor(),
    "Adaboost Regressor": AdaBoostRegressor(),
    "DecisionTree Regressor": DecisionTreeRegressor()
}
r2_scores_dict = {}
for i, j in models.items():
    c = j.fit(X_train, y_train)
    y_pred = c.predict(X_test)
    mae, mse, rmse, r2 = evaluate_model(y_test, y_pred)
    print(f"{i} - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R2: {r2}")
    r2_scores_dict[i] = r2

Linear Regression - MAE: 4.21476314247485, MSE: 29.095169866715487, RMSE: 5.393993869732843, R2: 0.8804332983749565
Lasso - MAE: 5.157881810347763, MSE: 42.5064168384116, RMSE: 6.519694535667419, R2: 0.8253197323627853
Ridge - MAE: 4.211100688014261, MSE: 29.056272192348324, RMSE: 5.390387016935642, R2: 0.8805931485028737
Random Forest - MAE: 4.635179166666666, MSE: 35.893519059027774, RMSE: 5.99112001707759, R2: 0.8524954587560871
KNN - MAE: 5.621, MSE: 52.6066, RMSE: 7.253040741647602, R2: 0.7838129945787431
XGBRegressor - MAE: 5.057730674743652, MSE: 41.90370559692383, RMSE: 6.47330716071189, R2: 0.8277965784072876
Adaboost Regressor - MAE: 4.814865691260245, MSE: 37.58976493972305, RMSE: 6.131049252756256, R2: 0.8455247304177103
DecisionTree Regressor - MAE: 6.3, MSE: 61.66, RMSE: 7.852388171760232, R2: 0.746608015833095


In [54]:
best_model = max(r2_scores_dict, key=r2_scores_dict.get)
print(f"Best model: {best_model} with R2 score: {r2_scores_dict[best_model]}")

Best model: Ridge with R2 score: 0.8805931485028737


In [55]:
worst_model = min(r2_scores_dict, key=r2_scores_dict.get)
print(f"Worst model: {worst_model} with R2 score: {r2_scores_dict[worst_model]}")

Worst model: DecisionTree Regressor with R2 score: 0.746608015833095
